In [ ]:
# importing libs
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda,Dense, GaussianNoise
from keras.models import Model
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam,SGD
import random as rn

In [ ]:
# defining parameters
M = 256
Mmod= 16
R = 1/2
k = int(np.log2(M))
kmod= int(np.log2(Mmod))
n_channel = int(k/R)
n= int(n_channel/kmod)

print ('M:',M,'k:',k,'n_coded:',n_channel,'n:',n,'R:',R,'M_mod:',Mmod)

M: 256 k: 8 n_coded: 16 n: 4 R: 0.5 M_mod: 16


In [ ]:
#generating data of size N
N = 1000000
label = np.random.randint(M,size=N)

In [ ]:
data = []
for i in label:
    temp=bin(i)[2:].zfill(k)
    temp=np.array([float(d) for d in temp])
    data.append(temp)

In [ ]:
data = np.array(data)
print (data.shape)

(1000000, 8)


In [ ]:
temp_check = [17,23,45,67,89,96,72,250,350]
for i in temp_check:
    print(label[i],data[i])

22 [0. 0. 0. 1. 0. 1. 1. 0.]
67 [0. 1. 0. 0. 0. 0. 1. 1.]
142 [1. 0. 0. 0. 1. 1. 1. 0.]
8 [0. 0. 0. 0. 1. 0. 0. 0.]
146 [1. 0. 0. 1. 0. 0. 1. 0.]
35 [0. 0. 1. 0. 0. 0. 1. 1.]
104 [0. 1. 1. 0. 1. 0. 0. 0.]
156 [1. 0. 0. 1. 1. 1. 0. 0.]
167 [1. 0. 1. 0. 0. 1. 1. 1.]


In [ ]:
lr=0.001
Eb_N0db=5
Eb_N0=10.0**(Eb_N0db/10.0)

In [ ]:
# defining autoencoder and it's layer
input_signal = Input(shape=(k,))
encoded = Dense(n_channel, activation='relu')(input_signal)
encoded1 = Dense(2*n, activation='relu')(encoded)
encoded2 = Dense(2*n, activation='relu')(encoded1)
encoded3 = Dense(2*n, activation='linear')(encoded2)
encoded4 = Lambda(lambda x: np.sqrt(2*n)*K.l2_normalize(x,axis=1))(encoded3)

encoded5 = GaussianNoise(np.sqrt(1/(2*R*kmod*Eb_N0)))(encoded4)

decoded = Dense(2*n, activation='relu')(encoded5)
decoded1 = Dense(n_channel, activation='linear')(decoded)
decoded2 = Dense(k, activation='sigmoid')(decoded1)

autoencoder = Model(input_signal, decoded2)
adam = Adam(lr)
autoencoder.compile(optimizer=adam, loss='binary_crossentropy')

In [ ]:
print (autoencoder.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense (Dense)                (None, 16)                144       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 72        
_________________________________________________________________
lambda (Lambda)              (None, 8)                 0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 8)                 0     

In [ ]:
autoencoder.fit(data, data,
                epochs=100,
                batch_size=1000)

Epoch 1/100
1000/1000 [==============================] - 3s 2ms/step - loss: 0.4901
Epoch 2/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.1861
Epoch 3/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1205
Epoch 4/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.1047
Epoch 5/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1011
Epoch 6/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0986
Epoch 7/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0964
Epoch 8/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0950
Epoch 9/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0941
Epoch 10/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0921
Epoch 11/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0804
Epoch 12/100
1000/1000 [==============================] - 3s 3ms/step - lo

In [ ]:
from keras.models import load_model
#autoencoder.save('4_7_symbol_autoencoder_v_best.model')

In [ ]:
#autoencoder_loaded = load_model('4_7_symbol_autoencoder_v_best.model')

In [ ]:
encoder = Model(input_signal, encoded4)

In [ ]:
# making decoder from full autoencoder
encoded_input = Input(shape=(2*n,))

deco = autoencoder.layers[-3](encoded_input)
deco1 = autoencoder.layers[-2](deco)
deco2 = autoencoder.layers[-1](deco1)
decoder = Model(encoded_input, deco2)

In [ ]:
N = 1000000
test_label = np.random.randint(M,size=N)
test_data = []

for i in test_label:
    temp=bin(i)[2:].zfill(k)
    temp=np.array([float(d) for d in temp])
    test_data.append(temp)
    
test_data = np.array(test_data)

In [ ]:
temp_test = 6
print (test_data[temp_test],test_label[temp_test])

[1. 1. 1. 1. 1. 1. 0. 0.] 252


In [ ]:
def frange(x, y, jump):
  while x < y:
    yield x
    x += jump

# BER

In [ ]:
EbNodB_range = list(frange(0,11,1))
ber = [None]*len(EbNodB_range)
for i in range(0,len(EbNodB_range)):
    EbNo=10.0**(EbNodB_range[i]/10.0)
    noise_std = np.sqrt(1/(2*R*kmod*EbNo))
    noise_mean = 0
    no_errors = 0
    noise = noise_std * np.random.randn(N,2*n)
    encoded_signal = encoder.predict(test_data) 
    final_signal = encoded_signal + noise
    pred_final_signal =  decoder.predict(final_signal)
    
    #pred_output = np.where(pred_final_signal >= 0.5 , 1, 0)
    ber[i] = K.mean(K.cast(K.not_equal(test_data, K.round(pred_final_signal)),dtype='float32')).numpy()
    print ('SNR:',EbNodB_range[i],'BER:',ber[i])

SNR: 0 BER: 0.062544
SNR: 1 BER: 0.044944376
SNR: 2 BER: 0.0300495
SNR: 3 BER: 0.018966125
SNR: 4 BER: 0.010895625
SNR: 5 BER: 0.00567525
SNR: 6 BER: 0.002659625
SNR: 7 BER: 0.001058125
SNR: 8 BER: 0.0003715
SNR: 9 BER: 0.000111875
SNR: 10 BER: 2.6125e-05


### Matlab Array form

In [ ]:
for i in range(0,len(EbNodB_range)):
  print(ber[i], " ",end='')

0.062544  0.044944376  0.0300495  0.018966125  0.010895625  0.00567525  0.002659625  0.001058125  0.0003715  0.000111875  2.6125e-05  

MER

In [ ]:
EbNodB_range = list(frange(0,11,1))
mer = [None]*len(EbNodB_range)
for i in range(0,len(EbNodB_range)):
    EbNo=10.0**(EbNodB_range[i]/10.0)
    noise_std = np.sqrt(1/(2*R*kmod*EbNo))
    noise_mean = 0
    no_errors = 0
    noise = noise_std * np.random.randn(N,2*n)
    encoded_signal = encoder.predict(test_data) 
    final_signal = encoded_signal + noise
    pred_final_signal =  decoder.predict(final_signal)
    #pred_output = np.where(pred_final_signal >= 0.5 , 1, 0)
    mer[i] = np.max((pred_final_signal.round()!=test_data),axis=1).mean()
    print ('SNR:',EbNodB_range[i],'mER:',mer[i])

SNR: 0 mER: 0.406486
SNR: 1 mER: 0.306428
SNR: 2 mER: 0.2168
SNR: 3 mER: 0.140864
SNR: 4 mER: 0.083349
SNR: 5 mER: 0.044436
SNR: 6 mER: 0.020562
SNR: 7 mER: 0.008475
SNR: 8 mER: 0.00304
SNR: 9 mER: 0.000873
SNR: 10 mER: 0.000198


In [ ]:
for i in range(0,len(EbNodB_range)):
  print(mer[i], " ",end='')

0.406486  0.306428  0.2168  0.140864  0.083349  0.044436  0.020562  0.008475  0.00304  0.000873  0.000198  

In [ ]:
np.max((pred_final_signal.round()!=test_data),axis=1).sum()/1000000

9.4e-05

Block Error rate

In [ ]:
EbNodB_range = list(frange(0,11,1))
bler = [None]*len(EbNodB_range)
for i in range(0,len(EbNodB_range)):
    EbNo=10.0**(EbNodB_range[i]/10.0)
    noise_std = np.sqrt(1/(2*R*kmod*EbNo))
    noise_mean = 0
    no_errors = 0
    noise = noise_std * np.random.randn(N,2*n)
    encoded_signal = encoder.predict(test_data) 
    final_signal = encoded_signal + noise
    pred_final_signal =  decoder.predict(final_signal)
    #pred_output = np.where(pred_final_signal >= 0.5 , 1, 0)
    predicted_block=pred_final_signal.reshape(10000,800)
    test_data_block=test_data.reshape(10000,800)
    bler[i] = np.max((predicted_block.round()!=test_data_block),axis=1).mean()
    print ('SNR:',EbNodB_range[i],'BLER:',bler[i])

SNR: 0 BLER: 1.0
SNR: 1 BLER: 1.0
SNR: 2 BLER: 1.0
SNR: 3 BLER: 1.0
SNR: 4 BLER: 0.9999
SNR: 5 BLER: 0.9873
SNR: 6 BLER: 0.8781
SNR: 7 BLER: 0.5753
SNR: 8 BLER: 0.2557
SNR: 9 BLER: 0.0819
SNR: 10 BLER: 0.0184


In [ ]:
for i in range(0,len(EbNodB_range)):
  print(bler[i], " ",end='')

1.0  1.0  1.0  1.0  0.9999  0.9873  0.8781  0.5753  0.2557  0.0819  0.0184  